<a href="https://colab.research.google.com/github/lorenzotagliapietra/ML_project_Tagliapietra/blob/main/Cluster_finder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

"""
# Clustering di Galassie in Diagramma Colore-Magnitudine

Questo notebook implementa una pipeline per identificare galassie compagne attorno a una radiogalassia centrale utilizzando metodi di clustering non supervisionato nel diagramma colore-magnitudine (CMD).
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
from sklearn.mixture import GaussianMixture
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, precision_score, recall_score
import seaborn as sns
from astropy.stats import sigma_clip

# Configurazione grafica
plt.style.use('seaborn')
sns.set_palette("colorblind")
%matplotlib inline

In [ ]:
"""
## 1. Caricamento e Preparazione dei Dati

Carichiamo i tre cataloghi disponibili:
1. Tutti gli oggetti nel campo
2. Solo oggetti classificati come galassie
3. Solo galassie ellittiche
"""


# Caricamento dei dati (sostituire con i percorsi reali dei file)
all_objects = pd.read_csv('catalogo_tutti.csv')
galaxies = pd.read_csv('catalogo_galassie.csv')
ellipticals = pd.read_csv('catalogo_ellittiche.csv')

# Visualizziamo la struttura dei dati
print("Tutti gli oggetti:")
display(all_objects.head())
print("\nGalassie:")
display(galaxies.head())
print("\nGalassie ellittiche:")
display(ellipticals.head())


In [ ]:
"""
## 2. Diagramma Colore-Magnitudine (CMD)

Creiamo il diagramma colore-magnitudine utilizzando le bande r e i:
- Asse X: colore (r - i)
- Asse Y: magnitudine (i)
"""


def plot_cmd(data, title, ax=None, color='b', alpha=0.5):
    """Plot del diagramma colore-magnitudine"""
    if ax is None:
        fig, ax = plt.subplots(figsize=(10, 8))

    ax.scatter(data['r'] - data['i'], data['i'],
               c=color, alpha=alpha, s=10, label=title)
    ax.set_xlabel('Colore (r - i)', fontsize=14)
    ax.set_ylabel('Magnitudine (i)', fontsize=14)
    ax.set_title(title, fontsize=16)
    ax.invert_yaxis()  # Le magnitudini più brillanti hanno valori più bassi
    ax.grid(True)
    return ax

# Plot per i tre cataloghi
fig, axes = plt.subplots(1, 3, figsize=(24, 8))
plot_cmd(all_objects, "Tutti gli oggetti", axes[0], color='b')
plot_cmd(galaxies, "Galassie", axes[1], color='g')
plot_cmd(ellipticals, "Galassie ellittiche", axes[2], color='r')
plt.tight_layout()
plt.show()


In [ ]:
"""
## 3. Clustering con Gaussian Mixture Models (GMM)

Applichiamo il GMM al diagramma CMD per identificare la red sequence.
"""

def fit_gmm(data, n_components=2, plot=True):
    """Applica GMM ai dati e visualizza i risultati"""
    # Preparazione dei dati
    X = data[['r-i', 'i']].values
    X[:, 0] = data['r'] - data['i']  # Calcolo del colore

    # Standardizzazione
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Fit del modello GMM
    gmm = GaussianMixture(n_components=n_components, covariance_type='full', random_state=42)
    gmm.fit(X_scaled)
    labels = gmm.predict(X_scaled)
    probs = gmm.predict_proba(X_scaled)

    # Calcolo di BIC e AIC
    bic = gmm.bic(X_scaled)
    aic = gmm.aic(X_scaled)

    if plot:
        # Plot dei risultati
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

        # Scatter plot con colori per cluster
        for i in range(n_components):
            mask = labels == i
            ax1.scatter(X[mask, 0], X[mask, 1], s=10, label=f'Cluster {i}', alpha=0.6)

        ax1.set_xlabel('Colore (r - i)', fontsize=14)
        ax1.set_ylabel('Magnitudine (i)', fontsize=14)
        ax1.set_title(f'GMM con {n_components} componenti', fontsize=16)
        ax1.invert_yaxis()
        ax1.legend()
        ax1.grid(True)

        # Plot delle ellissi delle gaussiane
        ax2.scatter(X[:, 0], X[:, 1], s=5, c='gray', alpha=0.3)

        for i in range(n_components):
            # Converti i parametri della gaussiana alla scala originale
            mean = scaler.inverse_transform(gmm.means_[i].reshape(1, -1)).flatten()
            cov = scaler.inverse_transform(gmm.covariances_[i]) * scaler.scale_ * scaler.scale_.reshape(-1, 1)

            # Calcola autovalori e autovettori per disegnare l'ellisse
            eigenvalues, eigenvectors = np.linalg.eigh(cov)
            angle = np.degrees(np.arctan2(*eigenvectors[:, 0][::-1]))
            width, height = 2 * np.sqrt(eigenvalues)

            ellipse = Ellipse(mean, width, height, angle=angle,
                             edgecolor='r', facecolor='none', linewidth=2)
            ax2.add_patch(ellipse)
            ax2.scatter(mean[0], mean[1], c='k', marker='x', s=100)

        ax2.set_xlabel('Colore (r - i)', fontsize=14)
        ax2.set_ylabel('Magnitudine (i)', fontsize=14)
        ax2.set_title(f'Contorni delle gaussiane', fontsize=16)
        ax2.invert_yaxis()
        ax2.grid(True)

        plt.show()

    return gmm, labels, probs, bic, aic

In [ ]:
# Applica GMM con diversi numeri di componenti
n_components_range = range(2, 6)
bics, aics = [], []

for n in n_components_range:
    print(f"\nGMM con {n} componenti:")
    gmm, labels, probs, bic, aic = fit_gmm(galaxies, n_components=n)
    bics.append(bic)
    aics.append(aic)

# Plot di BIC e AIC
plt.figure(figsize=(10, 6))
plt.plot(n_components_range, bics, 'o-', label='BIC')
plt.plot(n_components_range, aics, 'o-', label='AIC')
plt.xlabel('Numero di componenti', fontsize=14)
plt.ylabel('Valore del criterio', fontsize=14)
plt.title('Criteri BIC e AIC per selezione componenti GMM', fontsize=16)
plt.legend()
plt.grid(True)
plt.show()

# Seleziona il miglior modello in base a BIC
best_n = n_components_range[np.argmin(bics)]
print(f"\nNumero ottimale di componenti (basato su BIC): {best_n}")

# Fit del modello finale
final_gmm, final_labels, final_probs, _, _ = fit_gmm(galaxies, n_components=best_n)

# Identifica la componente della red sequence (assumendo sia la più rossa)
red_sequence_idx = np.argmax(final_gmm.means_[:, 0])  # Componente con colore medio più alto

# Seleziona membri del cluster con probabilità > 0.7
cluster_mask = final_probs[:, red_sequence_idx] > 0.7
cluster_members = galaxies[cluster_mask]

print(f"\nNumero di membri del cluster identificati: {len(cluster_members)}")

In [ ]:
"""
## 4. Confronto: GMM a 2 componenti

Testiamo anche un modello più semplice con solo 2 componenti per confronto.
"""

# Fit del modello a 2 componenti
gmm_2, labels_2, probs_2, _, _ = fit_gmm(galaxies, n_components=2)

# Identifica la componente della red sequence
red_sequence_idx_2 = np.argmax(gmm_2.means_[:, 0])

# Seleziona membri del cluster
cluster_mask_2 = probs_2[:, red_sequence_idx_2] > 0.7
cluster_members_2 = galaxies[cluster_mask_2]

print(f"\nNumero di membri del cluster con GMM a 2 componenti: {len(cluster_members_2)}")

# Confronto visivo
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20, 8))

# Plot del modello ottimale
plot_cmd(galaxies, "GMM ottimale", ax1, color='gray')
plot_cmd(cluster_members, f"Cluster (n={len(cluster_members)})", ax1, color='red')

# Plot del modello a 2 componenti
plot_cmd(galaxies, "GMM a 2 componenti", ax2, color='gray')
plot_cmd(cluster_members_2, f"Cluster (n={len(cluster_members_2)})", ax2, color='red')

plt.show()


In [ ]:
"""
## 5. Confronto con DBSCAN in RA/Dec

Applichiamo DBSCAN alle coordinate celesti per identificare overdensità spaziali.
"""

def apply_dbscan(data, eps=0.1, min_samples=5):
    """Applica DBSCAN alle coordinate RA/Dec"""
    # Estrai coordinate e normalizza
    X = data[['ra', 'dec']].values
    X = StandardScaler().fit_transform(X)

    # Applica DBSCAN
    db = DBSCAN(eps=eps, min_samples=min_samples).fit(X)
    labels = db.labels_

    # Numero di cluster (escludendo rumore)
    n_clusters = len(set(labels)) - (1 if -1 in labels else 0)

    # Plot dei risultati
    plt.figure(figsize=(10, 8))

    # Rumore
    mask = labels == -1
    plt.scatter(data['ra'][mask], data['dec'][mask],
                c='gray', alpha=0.3, s=10, label='Rumore')

    # Cluster
    for cluster_id in set(labels):
        if cluster_id == -1:
            continue
        mask = labels == cluster_id
        plt.scatter(data['ra'][mask], data['dec'][mask],
                    s=20, label=f'Cluster {cluster_id}', alpha=0.7)

    plt.xlabel('RA (deg)', fontsize=14)
    plt.ylabel('Dec (deg)', fontsize=14)
    plt.title(f'DBSCAN clustering (eps={eps}, min_samples={min_samples})', fontsize=16)
    plt.legend()
    plt.grid(True)

    return labels

# Applica DBSCAN alle galassie
dbscan_labels = apply_dbscan(galaxies)

# Seleziona il cluster principale (quello con più membri)
main_cluster_id = np.bincount(dbscan_labels[dbscan_labels != -1]).argmax()
main_cluster_mask = dbscan_labels == main_cluster_id
dbscan_cluster_members = galaxies[main_cluster_mask]

print(f"\nNumero di membri del cluster con DBSCAN: {len(dbscan_cluster_members)}")


In [ ]:
"""
## 6. Validazione con Redshift Spettroscopici

Utilizziamo i redshift spettroscopici disponibili per validare i risultati.
"""

# Assumiamo che la radiogalassia centrale abbia redshift z_RG
z_RG = 0.123  # Sostituire con il valore reale

# Carica i redshift spettroscopici (se disponibili)
try:
    z_spec = galaxies['z_spec'].values
    has_z_spec = ~np.isnan(z_spec)

    # Definisci i veri membri (entro delta_z < 0.01 dalla RG)
    true_members = np.abs(z_spec - z_RG) < 0.01

    # Confronta con i risultati del clustering
    gmm_predicted = np.zeros_like(true_members, dtype=bool)
    gmm_predicted[cluster_mask] = True

    dbscan_predicted = np.zeros_like(true_members, dtype=bool)
    dbscan_predicted[main_cluster_mask] = True

    # Metriche di performance
    def print_metrics(true, pred, method):
        cm = confusion_matrix(true, pred)
        precision = precision_score(true, pred)
        recall = recall_score(true, pred)

        print(f"\nMetriche per {method}:")
        print(f"Matrice di confusione:\n{cm}")
        print(f"Precision: {precision:.3f}")
        print(f"Recall: {recall:.3f}")
        print(f"Purezza: {cm[1,1]/(cm[0,1]+cm[1,1]):.3f}")
        print(f"Completezza: {cm[1,1]/(cm[1,0]+cm[1,1]):.3f}")

    print_metrics(true_members, gmm_predicted, "GMM")
    print_metrics(true_members, dbscan_predicted, "DBSCAN")

    # Plot dei redshift
    plt.figure(figsize=(12, 6))

    plt.hist(z_spec[has_z_spec], bins=30, alpha=0.7, label='Tutti')
    plt.hist(z_spec[gmm_predicted & has_z_spec], bins=30, alpha=0.7, label='GMM')
    plt.hist(z_spec[dbscan_predicted & has_z_spec], bins=30, alpha=0.7, label='DBSCAN')

    plt.axvline(x=z_RG, color='r', linestyle='--', label='Radiogalassia')
    plt.xlabel('Redshift', fontsize=14)
    plt.ylabel('Conteggio', fontsize=14)
    plt.title('Distribuzione dei redshift', fontsize=16)
    plt.legend()
    plt.grid(True)
    plt.show()

except KeyError:
    print("Nessun dato di redshift spettroscopico disponibile per la validazione.")


In [ ]:
"""
## 7. Approccio Ibrido

Combiniamo GMM nel CMD e DBSCAN in RA/Dec per una selezione più robusta.
"""

# Modello 1: GMM in CMD seguito da DBSCAN in RA/Dec
print("\nApproccio ibrido 1: GMM + DBSCAN")

# Prendiamo i candidati dal GMM
gmm_candidates = galaxies[cluster_mask]

# Applica DBSCAN a questi candidati
hybrid_labels = apply_dbscan(gmm_candidates, eps=0.05, min_samples=3)

# Seleziona il cluster principale
hybrid_cluster_id = np.bincount(hybrid_labels[hybrid_labels != -1]).argmax()
hybrid_cluster_mask = hybrid_labels == hybrid_cluster_id
hybrid_members = gmm_candidates[hybrid_cluster_mask]

print(f"Numero di membri con approccio ibrido: {len(hybrid_members)}")

# Modello 2: Clustering in spazio 4D (colore, magnitudine, RA, Dec)
print("\nApproccio ibrido 2: Clustering 4D")

# Prepara i dati
X_4d = galaxies[['r', 'i', 'ra', 'dec']].copy()
X_4d['r-i'] = X_4d['r'] - X_4d['i']
X_4d = X_4d[['r-i', 'i', 'ra', 'dec']].values

# Normalizza le feature
scaler_4d = StandardScaler()
X_4d_scaled = scaler_4d.fit_transform(X_4d)

# Applica GMM in 4D
gmm_4d = GaussianMixture(n_components=3, covariance_type='full', random_state=42)
gmm_4d.fit(X_4d_scaled)
labels_4d = gmm_4d.predict(X_4d_scaled)

# Seleziona il cluster più probabile per la red sequence
# (basato sulla media del colore e magnitudine)
means = scaler_4d.inverse_transform(gmm_4d.means_)
red_sequence_4d = np.argmax(means[:, 0])  # Componente con colore medio più alto
cluster_4d_mask = labels_4d == red_sequence_4d
cluster_4d_members = galaxies[cluster_4d_mask]

print(f"Numero di membri con clustering 4D: {len(cluster_4d_members)}")

# Confronto visivo
fig, axes = plt.subplots(1, 3, figsize=(24, 8))

# Plot CMD
plot_cmd(galaxies, "Tutti", axes[0], color='gray')
plot_cmd(hybrid_members, "GMM+DBSCAN", axes[0], color='blue')
plot_cmd(cluster_4d_members, "Clustering 4D", axes[0], color='green')

# Plot RA/Dec
axes[1].scatter(galaxies['ra'], galaxies['dec'], c='gray', alpha=0.3, s=5)
axes[1].scatter(hybrid_members['ra'], hybrid_members['dec'], c='blue', s=20, alpha=0.7)
axes[1].set_xlabel('RA (deg)', fontsize=14)
axes[1].set_ylabel('Dec (deg)', fontsize=14)
axes[1].set_title('Posizioni dei membri (GMM+DBSCAN)', fontsize=16)
axes[1].grid(True)

axes[2].scatter(galaxies['ra'], galaxies['dec'], c='gray', alpha=0.3, s=5)
axes[2].scatter(cluster_4d_members['ra'], cluster_4d_members['dec'], c='green', s=20, alpha=0.7)
axes[2].set_xlabel('RA (deg)', fontsize=14)
axes[2].set_ylabel('Dec (deg)', fontsize=14)
axes[2].set_title('Posizioni dei membri (Clustering 4D)', fontsize=16)
axes[2].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
"""
## Risultati Finali e Conclusioni

Sintetizziamo i risultati dei diversi approcci:
"""

# Tabella riassuntiva
results = pd.DataFrame({
    'Metodo': ['GMM ottimale', 'GMM 2 componenti', 'DBSCAN', 'GMM+DBSCAN', 'Clustering 4D'],
    'N_membri': [
        len(cluster_members),
        len(cluster_members_2),
        len(dbscan_cluster_members),
        len(hybrid_members),
        len(cluster_4d_members)
    ]
})

print("\nRisultati comparativi:")
display(results)

# Salva i risultati
cluster_members.to_csv('membri_cluster_gmm.csv', index=False)
hybrid_members.to_csv('membri_cluster_ibrido.csv', index=False)
cluster_4d_members.to_csv('membri_cluster_4d.csv', index=False)


"""
## Conclusioni

1. Il metodo GMM nel diagramma CMD è efficace per identificare la red sequence.
2. L'approccio ibrido (GMM + DBSCAN) fornisce una selezione più pulita combinando informazioni fotometriche e spaziali.
3. Il clustering in spazio 4D può essere potente ma più complesso da interpretare.
4. La validazione con redshift spettroscopici è cruciale per valutare la qualità della selezione.

Per ulteriori analisi:
- Sperimentare con diversi parametri per DBSCAN
- Provare altri algoritmi di clustering (es. HDBSCAN)
- Includere altre bande fotometriche per migliorare la separazione
"""